In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Convert data to numpy arrays
x_data = data[[ 'open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D



# Convert data to Tensors
x_feature_tensors = torch.tensor(x_data, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data, dtype=torch.float32)


# Split the data into training and testing sets
train_size = int(len(x_feature_tensors) * 0.67)
x_train, x_test = x_feature_tensors[:train_size], x_feature_tensors[train_size:]
y_train, y_test = y_feature_tensors[:train_size], y_feature_tensors[train_size:]



# Initialize the scaler
scaler = MinMaxScaler()

# fitting the MinMaxScaler to training data using fit_transform and then
#  test data only transform
x_train_scaled = scaler.fit_transform(x_train.detach().numpy())
x_test_scaled = scaler.transform(x_test.detach().numpy())  # Use transform, not fit_transform

y_train_scaled = scaler.fit_transform(y_train.detach().numpy())
y_test_scaled = scaler.transform(y_test.detach().numpy())  # Use transform, not fit_transform





In [3]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train_scaled) - window_size
batch_size = 1
input_size = x_train_scaled.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [7]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available

# Splitting the dataset into training, validation, and test sets
x_train, x_temp, y_train, y_temp = train_test_split(x_train_scaled, y_train_scaled, test_size=0.2, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, shuffle=False)

# Hyperparameter search space
search_space = {
    'learning_rate': [0.00001, 0.00005, 0.0001],
    'window_size': [3, 5, 6],
    'hidden_dim': [ 128, 256, 512],
    'n_layers': [10, 11, 12],
    'batch_evaluation_frequency': [2,3, 4]
}

Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 256,
                  'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 6463.20556640625}


# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    # Test set evaluation
    with torch.no_grad():
        x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)
        y_test_window = torch.tensor(y_test[:params['window_size']], dtype=torch.float32)
        x_test_window = x_test_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        test_outputs, _ = model(x_test_window, hidden)
        test_loss = criterion(test_outputs, y_test_window)

    print(f"Test Loss: {test_loss.item()}")

    # Update the best_params if the current model performs better on the test set
    if best_params is None:
        best_params = params
        best_params['test_loss'] = test_loss.item()
    elif test_loss < best_params['test_loss']:
        best_params = params
        best_params['test_loss'] = test_loss.item()

print(f"Best Parameters: {best_params}")

SyntaxError: invalid syntax (2082185232.py, line 24)

In [5]:
# ...

# Test set evaluation
with torch.no_grad():
    x_test_window = torch.tensor(x_test[-params['window_size']:], dtype=torch.float32)
    y_test_window = torch.tensor(y_test[-params['window_size']:], dtype=torch.float32)
    x_test_window = x_test_window.view(1, params['window_size'], input_size)

    hidden = model.init_hidden(1)
    test_outputs, _ = model(x_test_window, hidden)
    test_loss = criterion(test_outputs, y_test_window)

# Print the last entries of the last window
print(f"Last entries of the last window in the test set:")
print("Input values:")
print(x_test_window.squeeze().numpy())
print("Actual output values:")
print(y_test_window.squeeze().numpy())
print("Predicted output values:")
print(test_outputs.squeeze().numpy())

# ...


Last entries of the last window in the test set:
Input values:
[[0.66118854 0.6558897  0.65246195 0.19039789 0.6685383  0.5804368
  0.31263226]
 [0.6560159  0.65875053 0.66305935 0.1308405  0.67966837 0.62846404
  0.30835122]
 [0.6577598  0.6728215  0.6649329  0.15884562 0.69799155 0.64044154
  0.3110624 ]]
Actual output values:
[0.65143836 0.6631376  0.682414  ]
Predicted output values:
0.046670124


In [6]:
# Fit the scaler on the training data
scaler.fit(x_train.detach().numpy())

# Transform the training and test data using the fitted scaler
x_train_scaled = scaler.transform(x_train.detach().numpy())
x_test_scaled = scaler.transform(x_test.detach().numpy())

y_train_scaled = scaler.transform(y_train.detach().numpy())
y_test_scaled = scaler.transform(y_test.detach().numpy())

# Inverse transform the scaled test data to get the original values
x_test_original = scaler.inverse_transform(x_test_scaled)
y_test_original = scaler.inverse_transform(y_test_scaled)

# Extract the last entries of the last window
last_window_size = params['window_size']
x_test_last_window = x_test_original[-last_window_size:]
y_test_last_window = y_test_original[-last_window_size:]

# Print the last entries of the last window in the original scale
print("Last entries of the last window in the test set (original scale):")
print("Input values:")
print(x_test_last_window)
print("Actual output values:")
print(y_test_last_window)


AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [ ]:
# ... (Your model training and testing code here)

# Use the already fitted scaler to transform the test data
x_test_scaled = scaler.transform(x_test)  # Directly use NumPy array
y_test_scaled = scaler.transform(y_test)  # Directly use NumPy array

# Inverse transform the scaled test data to get the original values
x_test_original = scaler.inverse_transform(x_test_scaled)
y_test_original = scaler.inverse_transform(y_test_scaled)

# Extract the last entries of the last window
last_window_size = params['window_size']
x_test_last_window = x_test_original[-last_window_size:]
y_test_last_window = y_test_original[-last_window_size:]

# Print the last entries of the last window in the original scale
print("Last entries of the last window in the test set (original scale):")
print("Input values:")
print(x_test_last_window)
print("Actual output values:")
print(y_test_last_window)


NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.